In [1]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.SequencePreprocessing as sp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session
import random

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f5e7a03f880>
Traceback (most recent call last):
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
2024-03-07 18:41:03.421063: I tensorflow/core/pl

In [2]:
reload(test_utils)

<module 'test_utils' from '/home/ajp031/ClusterCast/test_utils.py'>

In [3]:
group_ids = [44]
groups = await test_utils.extract_cluster_groups(group_ids)

Creating and Processing Dataset
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
               macd  macd_signal  macd_diff
Date                                       
2007-01-03 -0.36177     0.002469  -0.441851
Empty DataFrame
Columns: []
Index: [2007-01-03 00:00:00]
              stoch_k    stoch_d
Date                            
2007-01-03  65.766718  60.755066
first date stoch: 2007-01-03 00:00:00
last date stoch: 2024-03-05 00:00:00
first date df: 2007-01-03 00:00:00
last date df: 2024-03-05 00:00:00
Scaling Quant Min Max Features
Quant Min Max Features Scaled
Scaling Standard
Name: momentum_vars len 10
Name: pctChg_vars len 115
Name: lag_features_vars len 230
Name: pctChgopen_rolling len 1
Name: pctChghigh_rolling len 1
Name: pctChglow_rolling len 1
Name: pctChgclose_rolling len 1
Name: pctChgvolume_rolling len 1
Name: pctChgVix_rolli

In [12]:
clusters = await test_utils.get_all_clusters(groups)

cur_group = groups[0]
features = groups[0].group_params.training_features

In [5]:
X_train, y_train, X_test, y_test = cur_group.get_3d_array() 

In [6]:
X_train = X_train.copy()[::-1]
y_train = y_train.copy()[::-1]
X_test = X_test.copy()[::-1]
y_test = y_test.copy()[::-1]

In [7]:
cluster_features = cur_group.group_params.cluster_features
X_feature_dict = cur_group.group_params.X_feature_dict
y_feature_dict = cur_group.group_params.y_feature_dict


In [9]:
clusters = test_utils.cluster_a_group(cur_group, (X_train, y_train, X_test, y_test), 6)

In [13]:
import random
target_features = [] 
# target_features += ['pctChgclose{}_target'.format(i) for i in range(-14, 0) ]
# target_features += ['pctChgclose-0_target']
target_features += ['pctChgclose+{}_target'.format(i) for i in range(1, 7) ]
training_features = random.sample(features, 30)
print(training_features)

['pctDiff+ema20_ema30', 'pctDiff+sma5_sma30', 'pctDiff+sma30_sma100', 'pctDiff+bb_high_close20', 'pctDiff+ema100_close', 'pctDiff+ema20_close', 'pctDiff+ema30_ema100', 'pctDiff+bb_high_low40', 'macd_signal', 'pctDiff+sma10_sma30', 'pctDiff+ema30_close', 'pctDiff+ema30_ema200', 'hiLo', 'pctChgclose', 'pctDiff+smaVol5_volume', 'macd_diff', 'pctDiff+sma5_sma20', 'pctDiff+ema10_ema200', 'pctDiff+bb_low_close10', 'rsi5', 'pctChgClLo', 'pctChgClHi', 'stoch_d', 'pctDiff+sma30_sma200', 'pctDiff+sma5_sma10', 'deriv+sma30', 'deriv+sma20', 'pctDiff+sma30_sma50', 'pctDiff+smaVol20_smaVol50', 'pctDiff+sma20_close']


In [ ]:
benchmark_model, test_model = test_utils.create_attention_model(input_steps=X_train.shape[1],output_steps=len(target_features),features=len(training_features))